# Area Aggregation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [1]:
import pandas as pd

In [6]:
df_play_metrics_all = pd.read_csv(f"{DIR}/data/outputs/play_metrics.csv")

(188254, 15)